In [20]:
%load_ext autoreload
%autoreload 2

In [21]:
import torch
from torch.utils.data import DataLoader
import torchvision
import matplotlib.pyplot as plt
import dataset
from model import ResNet18YOLOv1
from loss import YOLOv1Loss
from tqdm import tqdm

# About
This is an implementation of YOLOv1 from ***You Only Look Once: Unified, Real-Time Object Detection by Joseph Redmon, Santosh Divvala, Ross Girshick, and Ali Firhadi.*** Object detection is figuring out what objects are in an image and where they are. Another way to look at this problem is how can we write a computer program that draws bounding boxes around objects and predicts what kind of objects they are. YOLO solves this problem and does it super fast, like state of the art fast!

Let's talk about R-CNN, the predecessor to YOLO. It proposed regions, ran a classifier on every region, and did some post-processing to produce the final result. In simple language this translates to:
1. Lemme draw a lot of bounding boxes where I think objects are
2. Lemme figure out what are in the bounding boxes I drew
3. Ok, I drew too many bounding boxes, lemme remove most of them and keep the important ones

This is a lot of steps. What YOLO does instead is ***unified detection***. Unified detection combines the different components of object detection (where are the objects and what kind of objects are they) into one Convolutional Neural Network. You give it an image and in one swoop, it tells you exactly that.

Here's how it does it:
1. Divide the image into a SxS grid
2. Each cell in the grid predicts B bounding boxes and C class probabilities (what it thinks the object is)

We represent bounding boxes with 5 numbers: x, y, w, h, p.
- (x, y): center of the bounding box
- w: width
- h: height
- p: confidence (a measure of how confident we are that this box captures an object and matches the ground truth)

Accordingly, YOLOv1 produces a SxSx(5B+C) tensor. Each cell predicts B bounding boxes, how do we choose which one is the "true" predictor? How do we measure how good our bounding box and classification predictions are? 

We check which bounding box has the greatest overlap (IOU: Intersection Over Union) with the ground truth and choose that one as a predictor. We use this loss function to measure the "goodness" of our predictions:

![yolo loss function](https://i.stack.imgur.com/IddFu.png)

On a high level, it is the squared error between our prediction and the ground truth. 

# PASCAL VOC 2007 Dataset

PASCAL VOC Detection Dataset contains annotated images with 20 labelled classes and bounding boxes. There are 2,501 images in the training set, 2,510 images in the validation set, and 4,952 images in the test set.

In [22]:
# original dataset
pascal_voc_train = torchvision.datasets.VOCDetection(
    root="data",
    year="2007",
    image_set="train",
    download=False
)

pascal_voc_val = torchvision.datasets.VOCDetection(
    root="data",
    year="2007",
    image_set="val",
    download=True
)

pascal_voc_test = torchvision.datasets.VOCDetection(
    root="data",
    year="2007",
    image_set="test",
    download=True
)

Using downloaded and verified file: data/VOCtrainval_06-Nov-2007.tar
Extracting data/VOCtrainval_06-Nov-2007.tar to data
Using downloaded and verified file: data/VOCtest_06-Nov-2007.tar
Extracting data/VOCtest_06-Nov-2007.tar to data


In [23]:
# augment dataset for YOLOv1: resize and normalize image and convert bounding boxes from annotations to tensors
voc_train = dataset.PascalVOC(pascal_voc=pascal_voc_train)
voc_val = dataset.PascalVOC(pascal_voc=pascal_voc_val)
voc_test = dataset.PascalVOC(pascal_voc=pascal_voc_test)

TRANSFORMING PASCAL VOC
TRANSFORMING PASCAL VOC
TRANSFORMING PASCAL VOC


In [24]:
BATCH_SIZE = 64

In [25]:
train_dataloader = DataLoader(voc_train, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(voc_val, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(voc_test, batch_size=BATCH_SIZE, shuffle=True)

# Training

## Device

In [27]:
DEVICE = "cpu"

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    DEVICE = torch.device("mps")

DEVICE

device(type='cuda')

## Hyperparameters
- S: dimensions of SxS grid
- B: number of bounding boxes predicted per cell
- C: number of classes

In [28]:
S = 7
B = 2
C = 20
lambda_coord = 5.0
lambda_noobj = 0.5

## Model

In [29]:
yolo = ResNet18YOLOv1(S=S, B=B, C=C).to(DEVICE)

## Loss + Optimizer
![yolo loss function](https://i.stack.imgur.com/IddFu.png)

We use stochastic gradient descent with a learning rate of 1e-3, weight decay (L2 regularization) of 0.0005, and momentum of 0.9.

In [30]:
yolo_loss = YOLOv1Loss(S=S, B=B, C=C, lambda_coord=lambda_coord, lambda_noobj=lambda_noobj)
optimizer = torch.optim.SGD(yolo.parameters(), lr=1e-3, weight_decay=0.0005, momentum=0.9)

## Train

In [9]:
def train(model, criterion, optimizer, train_dataloader, val_dataloader):
    pass

In [10]:
EPOCHS = 10

In [11]:
losses = []

yolo.train()
for epoch in range(EPOCHS):
    total_loss = 0
    
    for i, (X, Y) in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch + 1} of {EPOCHS}", leave=False)):
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)
        
        pred = yolo(X)
        loss = yolo_loss(pred, Y)
        
        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        total_loss += loss.item()
    
    loss = total_loss / len(train_dataloader)
    losses.append(loss)
    
    print(f"Epoch [{epoch + 1}/{EPOCHS}], Train Loss: {loss}")

Epoch 1 of 1: 100%|█████████████████████████████| 40/40 [02:49<00:00,  4.23s/it]

Epoch [1/1], Train Loss: 31.714705514907838
